## Create Sampled Dataset of KITTI


In [5]:
import argparse
import os
import yaml
import numpy as np
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import concurrent.futures
import random
import time

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def load_sequence(dataroot, sequence):
    scan_paths = f'{dataroot}/{sequence}/velodyne'
    scan_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(
      os.path.expanduser(scan_paths)) for f in fn]
    scan_names.sort()
    
    label_paths = f'{dataroot}/{sequence}/labels'
    label_names = [os.path.join(dp, f) for dp, dn, fn in os.walk(
        os.path.expanduser(label_paths)) for f in fn]
    label_names.sort()    
    assert len(label_names) == len(scan_names)
    print(f'Found {len(scan_names)} pointclouds and labels for sequence {sequence}.')
    return scan_names, label_names

def load_config_file(config_file):    
    try:        
        CFG = yaml.safe_load(open(config_file, 'r'))        
        return CFG
    except Exception as e:
        print(e)        
        return None
    
def parse_calibration(dataroot, seq):
    filename = dataroot + '/' + seq + '/calib.txt'
    calib = {}
    calib_file = open(filename)
    for line in calib_file:
        key, content = line.strip().split(":")
        values = [float(v) for v in content.strip().split()]

        pose = np.zeros((4, 4))
        pose[0, 0:4] = values[0:4]
        pose[1, 0:4] = values[4:8]
        pose[2, 0:4] = values[8:12]
        pose[3, 3] = 1.0

        calib[key] = pose
    calib_file.close()
    return calib

# dataroot = '/mnt/data/datasets/KITTI/sequences'
# dataroot = '/media/berlukas/Data/data/datasets/KITTI/sequences'
dataroot = '/media/berlukas/SSD_1TB/data_odometry_velodyne/dataset/sequences'
# dataroot = '/media/scratch/berlukas/kitti/sequences'

# sequences = ['01', '02', '03']
sequences = ['10']
config_file = '../config/semantic-kitti.yaml'
# export_ds = '/media/berlukas/Data/data/nuscenes'
# export_ds = '/media/scratch/berlukas/kitti/processed'
export_ds = '/media/berlukas/SSD_1TB/s2ae/kitti'

print(f'Setting dataroot to {dataroot}.')
print(f'Setting export path to {export_ds}.')
print(f'Configured {len(sequences)} sequences.')
print(f'Configured config file {config_file}')

Setting dataroot to /media/berlukas/SSD_1TB/data_odometry_velodyne/dataset/sequences.
Setting export path to /media/berlukas/SSD_1TB/s2ae/kitti.
Configured 1 sequences.
Configured config file ../config/semantic-kitti.yaml


In [3]:
all_sem_clouds = []

def parse_poses(dataroot, seq, calib):
    file = dataroot + '/' + seq + '/poses.txt'
    poses_arr = pd.read_csv(file, delimiter=' ', comment='#', header=None).to_numpy()    
    poses = [np.array([[r[0], r[1], r[2], r[3]],
                   [r[4], r[5], r[6], r[7]],
                   [r[8], r[9], r[10], r[11]],
                   [0, 0, 0, 1]]) for r in poses_arr]
    
    T_C_L = calib['Tr']
    n_poses = len(poses)
    for i in range(0, n_poses):    
        T_G_C = poses[i]
        poses[i] = T_G_C @ T_C_L
    return poses

def get_pointcloud_at(scan, name, label):
    pointclouds = []            
    scan.open_scan(name)
    scan.open_label(label)
    scan.colorize()

    pc = np.column_stack((scan.points, scan.remissions, scan.sem_label))
#     mask = pc[:,4] > 0 # Filter based on labeled data.        
    pointclouds.append(pc)
    return pointclouds

def get_map_at(scan, names, labels, indices):
    pointclouds = []    
    for idx in indices:
        scan.open_scan(names[idx])
        scan.open_label(labels[idx])
        scan.colorize()
        
        pc = np.column_stack((scan.points, scan.remissions, scan.sem_label))
        mask = pc[:,4] > 0 # Filter based on labeled data.        
        pointclouds.append(pc[mask])
    return pointclouds

def retrieve_poses_at(all_poses, indices):
    poses = []
    for idx in indices:
        poses.append(all_poses[idx])
    return poses

def combine_pointclouds(pointclouds, poses):
    n_data = len(poses)
    
    pivot = n_data // 2  
    T_G_L_pivot = poses[pivot]
    T_L_pivot_G = np.linalg.inv(T_G_L_pivot)

    acc_points = pointclouds[pivot]
    for i in range(0, n_data):
        if i == pivot:
            continue

        T_G_L = poses[i]
        T_L_pivot_L = T_L_pivot_G @ T_G_L

        points = Utils.transform_pointcloud(pointclouds[i], T_L_pivot_L)
        acc_points = np.append(acc_points, points, axis=0)                    
    
    return acc_points

CFG = load_config_file(config_file)
color_dict = CFG["color_map"]
nclasses = len(color_dict)
scan = SemLaserScan(nclasses, color_dict, project=False)
assert CFG is not None
  

for seq in sequences:
    print(f'Loading sequence {seq}.')    
    scan_names, label_names = load_sequence(dataroot, seq)
    calib = parse_calibration(dataroot, seq)
    poses = parse_poses(dataroot, seq, calib)    
    n_scans = len(scan_names)    
    print(f'This sequence has {len(poses)} data elements.')
        
    all_sem_clouds = []
    for i in tqdm(range(0, n_scans)):
        pointcloud = get_pointcloud_at(scan, scan_names[i], label_names[i])        
        # all_sem_clouds.append(Sphere(pointcloud[0]))
        all_sem_clouds.append(pointcloud[0])        

Loading sequence 10.
Found 1201 pointclouds and labels for sequence 10.
This sequence has 1201 data elements.


  0%|          | 0/1201 [00:00<?, ?it/s]

In [4]:
# Resample archive
filename = f'archive08'
archive = f'{export_ds}/{filename}.npy'
all_sem_clouds = np.load(archive, allow_pickle=True)
print(f'Loaded {len(all_sem_clouds)} pointclouds from {archive}')

Loaded 4071 pointclouds from /media/berlukas/SSD_1TB/s2ae/kitti/archive08.npy


In [4]:
np.random.shuffle(all_sem_clouds)
all_sem_clouds = all_sem_clouds[0:500]

In [9]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):
    sample = all_sem_clouds[sample_idx]
    sample_sphere = Sphere(sample)
#     sample_sphere = all_sem_clouds[sample_idx]
    features = sample_sphere.sampleUsingGrid(grid)
    return features

bw = 50
grid, _ = DHGrid.CreateGrid(bw)
print(f"Loading complete. Computing features...")
# parallel
sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
start_time = time.time() * 1000
sem_features = process_map(sample_func, sem_idx, max_workers=16, chunksize=50)
executionTime = (time.time() * 1000 - start_time)
print(f"It took {executionTime / len(all_sem_clouds)} ms.")

# filename = f"{export_ds}/clouds-{seq}.npy"
# np.save(filename, sem_features)
# print(f"Wrote features to {filename}.")

Loading complete. Computing features...


  0%|          | 0/1201 [00:00<?, ?it/s]

It took 28.77296832717527 ms.


In [5]:
filename = f"{export_ds}/clouds-10-full.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

Wrote features to /media/berlukas/SSD_1TB/s2ae/kitti/clouds-10-full.npy.


In [6]:
filename = f'archive{seq}'
np.save(f'{export_ds}/{filename}.npy', all_sem_clouds)
print(f'Saved clouds to {export_ds}/{filename}.npy')

/usr/lib/python3/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Saved clouds to /media/berlukas/SSD_1TB/s2ae/kitti/archive05.npy


---------------------------------------------------------

In [3]:
filename = f"{export_ds}/clouds-00.npy"
sem_features = np.load(filename)
print(f"Found {sem_features.shape[0]} features.")

Found 1500 features.


--------------------------------

In [ ]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd])    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

In [ ]:
pc = all_sem_clouds[0].point_cloud
visualizeRawPointcloud(pc, pc[:, 4])

In [ ]:
sem_features[0,:,:,:].shape

In [ ]:
filename = f"{export_ds}/clouds-08-2.npy"
sem_features = np.load(filename)


cur_sem_cloud = sem_features[0, :, :, :]
cur_sem_cloud = np.reshape(cur_sem_cloud, (3, -1)).T
print(f'{cur_sem_cloud.shape}')
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
print(f"sampling pointcloud shape is {points_xyz.shape}")
print(f"feature shape is {cur_sem_cloud.shape}")
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,2]))
# points_xyzn = np.column_stack((points_xyz, cur_sem_cloud[:,2]))

visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])